## Setup and Imports


In [14]:
import os
import numpy as np
import pandas as pd
import librosa
import soundfile as sf
from scipy.signal import fftconvolve
from scipy.io import loadmat

print("✅ Libraries loaded.")

✅ Libraries loaded.


## File Path Configuration


In [34]:
# Set your file paths
dry_audio_dir = ""
output_dir = ""
csv_path = "" # Estimated Acoustic Parameters CSV File Path
hrir_path = "" # HRIR Folder Path

os.makedirs(output_dir, exist_ok=True)
print("✅ Paths set.")

✅ Paths set.


## Loading HRIR Data


In [36]:
data = loadmat(hrir_path)
hrir_l = data['hrir_l']
hrir_r = data['hrir_r']

azimuths = np.array([-80.0, -65.0, -55.0, -45.0, -40.0, -35.0, -30.0, -25.0, -20.0, -15.0,
                     -10.0,  -5.0,   0.0,   5.0,  10.0,  15.0,  20.0,  25.0,  30.0,  35.0,
                      40.0,  45.0,  55.0,  65.0,  80.0])
azimuths = np.concatenate([azimuths[::-1] * -1, azimuths])[:50]
elevations = np.linspace(-45, 230.625, 25)

az_idx = np.argmin(np.abs(azimuths - 0.0))
el_idx = np.argmin(np.abs(elevations - 0.0))

hrir_left = hrir_l[el_idx, az_idx]
hrir_right = hrir_r[el_idx, az_idx]

print("✅ HRIR loaded for azimuth=0°, elevation=0°")

✅ HRIR loaded for azimuth=0°, elevation=0°


## Function Definition: generate_exponential_rir


In [38]:
def generate_exponential_rir(rt60, sr, length=1.5):
    n_samples = int(length * sr)
    t = np.linspace(0, length, n_samples)
    rir = np.exp(-6.91 * t / rt60)
    return rir / np.max(np.abs(rir))

def apply_rt60_reverb(signal, sr, rt60):
    rir = generate_exponential_rir(rt60, sr)
    return fftconvolve(signal, rir, mode='full')[:len(signal)]

def mix_drr(dry, reverb, drr_db):
    drr_linear = 10 ** (drr_db / 20)
    return dry + (reverb / drr_linear)

def render_binaural_rt60_drr(dry_audio, sr, rt60, drr):
    reverb = apply_rt60_reverb(dry_audio, sr, rt60)
    mixed = mix_drr(dry_audio, reverb, drr)
    left = fftconvolve(mixed, hrir_left, mode='full')[:len(mixed)]
    right = fftconvolve(mixed, hrir_right, mode='full')[:len(mixed)]
    binaural = np.stack([left, right], axis=1)
    return binaural / np.max(np.abs(binaural))

## Batch Processing and Rendering


In [42]:
df = pd.read_csv(csv_path)

# Select top 50 predicted rows (or fewer if you want)
df = df.head(50)

dry_files = sorted([f for f in os.listdir(dry_audio_dir) if f.endswith(".wav")])
dry_files = dry_files[:len(df)]  # Match number of rows exactly

print(f"✅ Found {len(dry_files)} dry files and {len(df)} parameter rows")

for i, filename in enumerate(dry_files):
    dry_path = os.path.join(dry_audio_dir, filename)
    dry_audio, sr = librosa.load(dry_path, sr=16000)

    row = df.iloc[i]
    rt60 = row['RT60']
    drr = row['DRR']
    c50 = row['C50']  # Not used

    print(f"🎧 [{i+1}/{len(dry_files)}] Processing {filename} | RT60={rt60:.2f}, DRR={drr:.2f}")
    rendered = render_binaural_rt60_drr(dry_audio, sr, rt60, drr)

    out_path = os.path.join(output_dir, f"rendered_{i:03d}_{filename}")
    sf.write(out_path, rendered, sr)
    print(f"✅ Saved to {out_path}")

✅ Found 50 dry files and 50 parameter rows
🎧 [1/50] Processing sample_000.wav | RT60=0.56, DRR=-0.66
✅ Saved to /Users/petros/MSc Computer Science/EECS MSc Project 24-25/Approach_1/rendered_batch_outputs/rendered_000_sample_000.wav
🎧 [2/50] Processing sample_001.wav | RT60=0.87, DRR=-3.32
✅ Saved to /Users/petros/MSc Computer Science/EECS MSc Project 24-25/Approach_1/rendered_batch_outputs/rendered_001_sample_001.wav
🎧 [3/50] Processing sample_002.wav | RT60=0.78, DRR=0.89
✅ Saved to /Users/petros/MSc Computer Science/EECS MSc Project 24-25/Approach_1/rendered_batch_outputs/rendered_002_sample_002.wav
🎧 [4/50] Processing sample_003.wav | RT60=0.75, DRR=-7.55
✅ Saved to /Users/petros/MSc Computer Science/EECS MSc Project 24-25/Approach_1/rendered_batch_outputs/rendered_003_sample_003.wav
🎧 [5/50] Processing sample_004.wav | RT60=0.70, DRR=1.19
✅ Saved to /Users/petros/MSc Computer Science/EECS MSc Project 24-25/Approach_1/rendered_batch_outputs/rendered_004_sample_004.wav
🎧 [6/50] Proce